In [5]:
import requests
import pandas as pd

In [59]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline
#zuerst die nötigen Biblis runterladen.

In [60]:
r=requests.get("https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page=1&utf8=✓")
soup=BeautifulSoup(r.text, "html.parser")

In [61]:
soup

<!DOCTYPE html>

<html>
<head>
<title>Geschäftsverzeichnis - Kanton Zug</title>
<link data-turbolinks-track="true" href="/assets/application-12e2e9430aa9f92b6bba06915b9032a2d09ab6904f35afd84ac9a39000ed16a6.css" media="all" rel="stylesheet"/>
<link data-turbolinks-track="true" href="/assets/print_landscape-c2f30d73eceed6ecc04fa9579abdf9aeb7e6e56f6ae705e5dd3c652fb0765daa.css" media="all" rel="stylesheet"/>
<script data-turbolinks-track="true" src="/assets/application-324c0218a21c03eecb7c8c3d3c99c429b72f1f34ac943b12deb044d2088626e4.js"></script>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="wgQg5dHK5cI/D5ceCKoQQGDyuzo/E2+1kP1O6ZBYhVAO+bTnpPFVNniXYubWuU3Hsysl3WipiKkJdLJb5GgaBw==" name="csrf-token"/>
<link href="/assets/favicon-931a70e08baa927b26976c4f47cfb20cdfec6d261f10a967aa0e43eeb3a82d5d.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
</head>
<body>
<div cl

In [62]:
tr_list=soup.find_all("tr") #das ist eine Liste mit allen "tr"-Tags drinn.

In [63]:
tr_list[3]#durchschauen, ab welcher Position es spannend wird und die Daten sind die wir baruchen.

<tr>
<td><a href="/gast/geschaefte/1997">3009</a></td>
<td><a href="/gast/geschaefte/1997">Postulat von Stéphanie Vuichard, Mariann Hess und Rita Hofer betreffend Biodiversitätsförderung</a></td>
<td class="hidden-xs">Überweisung an Regierungsrat</td>
<td class="hidden-xs">Baudirektion</td>
<td class="hidden-xs">Postulat</td>
<td class="hidden-xs">hängig</td>
</tr>

In [64]:
tr_liste=tr_list[3:] #da die Daten erst ab der 3. Position verwendet werden sollen, speichere ich die in neue Variable.

In [65]:
tr_liste[0].find_all("td")[1].text #nun haben wir alle td rausgeholt und sie in einer Liste. 
#dort schauen wir die zweite Position an. Mit .text bringen wir dort den Text alleine raus. 

'Postulat von Stéphanie Vuichard, Mariann Hess und Rita Hofer betreffend Biodiversitätsförderung'

In [66]:
td_list=tr_liste[0].find_all("td") #nun eine vorübergehende Variable besteimmen. hier temp

In [67]:
for td_tag in td_list:
    print(td_tag.text)

3009
Postulat von Stéphanie Vuichard, Mariann Hess und Rita Hofer betreffend Biodiversitätsförderung
Überweisung an Regierungsrat
Baudirektion
Postulat
hängig


In [68]:
td_list[0]

<td><a href="/gast/geschaefte/1997">3009</a></td>

In [70]:
#nun müssen wir noch an den Titel rankommen. Der kommt nicht raus, weil er nicht als Text drinn gespeichert ist.
#denn .text sieht nur das was auf der Page ersichtlich ist.
td_list[0].text

'3009'

In [46]:
td_list[0].find("a")["href"] #href ist wie ein key. das a müssen wir auslese, da es eine unterkathegorie von td ist.
#alle html-Tags die eine weiter funktion bekanntgeben ist wie ein key 

'/gast/geschaefte/1334'

In [51]:
#hier würde auch die Geschäftsnummer drinn stehen. Nur ist diese für mich gerade nicht erreichbar.
td_list[0].find("a")

<a href="/gast/geschaefte/1334">81</a>

In [56]:
#nun müssen wir Infos zusammenziehen
#zuerst ein Minidic bauen
#wir ziehen nun ein Element nach dem anderen raus.
td_list = tr_liste[0].find_all("td")
vorlag_nr=td_list[0].text
url = td_list[0].find("a")["href"]
text = td_list[1].text
stand = td_list[2].text
zustaendig = td_list[3].text
geschaeft_art = td_list[4].text
status = td_list[5].text

#weil es relativ übersichtlich ist, haben wir jede Variable einzel definiert und geschrieben. Sonst wärs Loop in loop.
#nun packen wir es in minidict

mini_dict = {"GeschäftsNR": vorlag_nr, "url":url,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}


In [71]:
#jetzt müssen wir dies auf alle "tr"-Positionen (gute_liste) auf der Seite 1 anwenden.
# die tr_liste durch element ersetzen. Bei Element keine "ortsangabe", da es bereits sagt, wo es  ist.
seite_eins = []
for element in tr_liste:
    td_list = element.find_all("td")
    vorlag_nr=td_list[0].text
    url = td_list[0].find("a")["href"]
    text = td_list[1].text
    stand = td_list[2].text
    zustaendig = td_list[3].text
    geschaeft_art = td_list[4].text
    status = td_list[5].text

#weil es relativ übersichtlich ist, haben wir jede Variable einzel definiert und geschrieben. Sonst wärs Loop in loop.
#nun packen wir es in minidict

    mini_dict = {"GeschäftsNR": vorlag_nr, "Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

    seite_eins.append(mini_dict)


In [72]:
pd.DataFrame(seite_eins)

,Art des Geschäfts,Geschäft,GeschäftsNR,Stand,Status,Zuständigkeit
0,Postulat,"Postulat von Stéphanie Vuichard, Mariann Hess ...",3009,Überweisung an Regierungsrat,hängig,Baudirektion
1,Motion,Motion von Rainer Leemann und Michael Arnold b...,3008,Überweisung an Regierungsrat,hängig,Gesundheitsdirektion
2,Interpellation,Interpellation der SP-Fraktion betreffend Pest...,3007,Überweisung an Regierungsrat,hängig,Baudirektion
3,Interpellation,Interpellation von Pirmin Andermatt und Heini ...,3006,Überweisung an Regierungsrat,hängig,Finanzdirektion
4,Wahlen,Kantonsrats-Ersatzwahl in der Einwohnergemeind...,3005,Wahl: Feststellung Gültigkeit,abgeschlossen,Staatskanzlei
5,Motion,Motion der CVP-Fraktion betreffend bedarfsgere...,3004,Überweisung an Regierungsrat,hängig,Direktion für Bildung und Kultur
6,Postulat,"Postulat von Manuela Leemann, Benny Elsener un...",3003,Überweisung an Regierungsrat,hängig,Baudirektion
7,Interpellation,"Interpellation von Hans Baumgartner, Manuela K...",3002,Überweisung an Regierungsrat,hängig,Baudirektion
8,Wahlen,Kantonsrats-Ersatzwahl in der Einwohnergemeind...,3001,Wahl: Feststellung Gültigkeit,abgeschlossen,Staatskanzlei
9,Interpellation,"Interpellation von Pirmin Andermatt, Zari Dzaf...",3000,Überweisung an Regierungsrat,hängig,Baudirektion


In [75]:
#jetzt machen wir das für alle Seiten. (aber noch nicht ganz automatisiert)
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="
#zuerst wollen wir die Seite besuchen. 
#nun müssen wir das machen was wir ganz am Anfang gemacht haben, aber im Forloop.Durch alle 40 Seiten.
# danach müssen wir den anderen Loop ranhängen. Sodass er es für alle 40 Seiten macht.
# wir machen dazu auch eine neue Liste für alle Seiten.
alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] # 2 weil der Kopf (die ersten zwei Positionen) uns nicht interessiert
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"GeschäftsNR": vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [78]:
df_a=pd.DataFrame(alle_seiten)

In [19]:
#Hier der Versuch, den "unterloop" als Definition zu machen.
def minidict(listenelement):
 

    for element in listenelement:
        td_list = element.find_all("td")
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        return mini_dict

In [20]:
df=pd.DataFrame(alle_seiten)

In [21]:
df["url"]
#so kann ich einzelne Sachen rausziehen.

0       /gast/geschaefte/1998
1       /gast/geschaefte/1997
2       /gast/geschaefte/1996
3       /gast/geschaefte/1995
4       /gast/geschaefte/1994
5       /gast/geschaefte/1993
6       /gast/geschaefte/1992
7       /gast/geschaefte/1991
8       /gast/geschaefte/1990
9       /gast/geschaefte/1989
10      /gast/geschaefte/1988
11      /gast/geschaefte/1987
12      /gast/geschaefte/1986
13      /gast/geschaefte/1985
14      /gast/geschaefte/1984
15      /gast/geschaefte/1983
16      /gast/geschaefte/1982
17      /gast/geschaefte/1981
18      /gast/geschaefte/1979
19      /gast/geschaefte/1980
20      /gast/geschaefte/1978
21      /gast/geschaefte/1977
22      /gast/geschaefte/1976
23      /gast/geschaefte/1975
24      /gast/geschaefte/1974
25      /gast/geschaefte/1973
26      /gast/geschaefte/1972
27      /gast/geschaefte/1971
28      /gast/geschaefte/1970
29      /gast/geschaefte/1969
                ...          
1933    /gast/geschaefte/1335
1934    /gast/geschaefte/1333
1935    /g

In [22]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df["url"]:
    url_liste_ganz.append(url_anfang_original+element)
    
 

In [23]:
url_liste_ganz

['https://kr-geschaefte.zug.ch/gast/geschaefte/1998',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1997',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1996',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1995',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1994',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1993',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1992',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1991',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1990',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1989',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1988',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1987',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1986',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1985',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1984',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1983',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1982',
 'https://kr-geschaefte.zug.ch/gast/geschaefte/1981',
 'https://kr-geschaefte.zug.

In [24]:
#Auf diese URL-s können wir unser System wieder draufschicken.

## Nun gehts in die Unterseiten. 
Dort von diesen erstelle ich ein neues Dataframe mit Geschäftsnummer und Einreichedatum

In [25]:
rg=requests.get("https://kr-geschaefte.zug.ch/gast/geschaefte/1997")

In [26]:
soup_g=BeautifulSoup(rg.text, "html.parser")

In [27]:
soup_g

<!DOCTYPE html>

<html>
<head>
<title>Geschäftsverzeichnis - Kanton Zug</title>
<link data-turbolinks-track="true" href="/assets/application-12e2e9430aa9f92b6bba06915b9032a2d09ab6904f35afd84ac9a39000ed16a6.css" media="all" rel="stylesheet"/>
<link data-turbolinks-track="true" href="/assets/print_landscape-c2f30d73eceed6ecc04fa9579abdf9aeb7e6e56f6ae705e5dd3c652fb0765daa.css" media="all" rel="stylesheet"/>
<script data-turbolinks-track="true" src="/assets/application-324c0218a21c03eecb7c8c3d3c99c429b72f1f34ac943b12deb044d2088626e4.js"></script>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="9YnJmvNS8VoiJxwSh0cKImuAURYU/hd/8Inn+59rykQn9PE1XHSHZI3jcPQbH5w+CgzjpiSk947eBqv2k5222g==" name="csrf-token"/>
<link href="/assets/favicon-931a70e08baa927b26976c4f47cfb20cdfec6d261f10a967aa0e43eeb3a82d5d.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
</head>
<body>
<div cl

In [28]:
tr_g_list=soup_g.find_all("tr")

In [29]:
tr_g_list

[<tr>
 <th class="col-lg-3">Geschäftsart</th>
 <td class="col-lg-9">Postulat</td>
 </tr>, <tr>
 <th class="col-lg-3">Eingereicht am</th>
 <td class="col-lg-9 geschaeft-eingereicht_am">10.09.2019</td>
 </tr>, <tr>
 <th class="col-lg-3">Eingereicht von</th>
 <td class="col-lg-9">
 <ul class="nomargin list-unstyled" id="eingereicht-von">
 <li>
 <a href="https://zg-comp.clex.ch/frontend/people/164">Vuichard Stéphanie</a>
 </li>
 <li>
 <a href="https://zg-comp.clex.ch/frontend/people/116">Hess Mariann</a>
 </li>
 <li>
 <a href="https://zg-comp.clex.ch/frontend/people/98">Hofer Rita</a>
 </li>
 </ul>
 </td>
 </tr>, <tr>
 <th class="col-lg-3">Abgeschlossen am</th>
 <td class="col-lg-9 geschaeft-abgeschlossen_am"></td>
 </tr>, <tr>
 <th class="col-lg-3">Status</th>
 <td class="col-lg-9 geschaeft-status">hängig</td>
 </tr>, <tr>
 <th class="col-lg-3">Zuständig</th>
 <td class="col-lg-9 geschaeft-zustaendig">Baudirektion</td>
 </tr>, <tr>
 <th class="col-lg-3">Verfahrensstand</th>
 <td class="co

In [30]:
#hier nochmals die Nummer des Geschäfts
tr_g_list[-1].find_all("td")[0].text

'3009'

In [31]:
#Und hier das Einreichedatum
tr_g_list[1].find_all("td")[0].text

'10.09.2019'

In [32]:
#Nun baue ich zuerst ein Minidict.
gesch_nr = tr_g_list[-1].find_all("td")[0].text
einger_am = tr_g_list[1].find_all("td")[0].text

minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am}


In [33]:
#For-loop schreiben um Seite für Seite abzurufen und deren Inhalte in das minidict und dieses in eine Liste packen
eingang_list=[]

for geschaeft in url_liste_ganz:
    rg=requests.get(geschaeft)
    soup_g=BeautifulSoup(rg.text, "html.parser")
    tr_g_list=soup_g.find_all("tr")
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text
      
    
    minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
            eingang_list.append(minidict_g)


In [34]:
df_g=pd.DataFrame(eingang_list)

In [3]:
#ah, da liegt also das Problem, dass ich nicht plotten kann. es sind biedes Objekte. muss es also umwandeln. hmmm w
pd.set_option("display.max_columns",None)

NameError: name 'pd' is not defined

In [36]:
df_g

,Einreichedatum,GeschäftsNR
0,10.09.2019,3010
1,10.09.2019,3010
2,10.09.2019,3009
3,10.09.2019,3009
4,10.09.2019,3008
5,10.09.2019,3008
6,08.09.2019,3007
7,08.09.2019,3007
8,04.09.2019,3006
9,04.09.2019,3006


In [37]:
df_g.dtypes

Einreichedatum    object
GeschäftsNR       object
dtype: object

In [52]:
#Zwischenstand: nun hat er zwar aus allen leeren Feldern ein NaN gemacht. 
#Trotzdem betrachtet er die Daten noch als Objekte. 

#Ich versuche mal weiter zu machen, indem ich die beiden Dataframes zusammentun
 

In [80]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR")

In [81]:
df_mit_dat.dtypes

Art des Geschäfts    object
Geschäft             object
GeschäftsNR          object
Stand                object
Status               object
Zuständigkeit        object
url                  object
Einreichedatum       object
dtype: object

In [ ]:
#geschafft. nun sind zumindest diese beiden Dataframes vereint. 
# Jetzt muss ich es noch hinkriegen, dass die Objekte zu verwertbaren Zahlen werden.

In [6]:
#nun der Versuch, die ganze Liste zu sehen.
df_g=pd.DataFrame(eingang_list)
pd.set_option("display.max_rows", 3000)

NameError: name 'eingang_list' is not defined

In [7]:
#ahhhh, es sind alles Strings. jetzt muss ich die in Daten und Nummern verwandeln
eingang_list

NameError: name 'eingang_list' is not defined

In [ ]:
df_g["GeschäftsNR"]= df_g["GeschäftsNR"].astype(int) #die Vorlagenummer als Zahle

In [ ]:
df_g.dtypes

In [ ]:
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #hier noch das Einreichedatum als Zahl.

In [ ]:
df_g.dtypes